In [1]:
import os
import requests
import pandas as pd

In [2]:
URID_location = pd.read_csv("../../System_Recovery/webapp/data/qc_streaming_DEMO.csv")
URID_location.head()

,Run,ProviderId,EvOrder,EvId,Activity,ETA,DwellTime,StreetNo,OnStreet,City,...,BookingType,FundingSourceId1,PassOn,PassOff,ClientId,SpaceOn,SpaceOff,ReqLate,SchTime,MobAids
0,180SEB,5.0,1.0,1145.0,4,42418.0,0.0,2000,118TH AVE SE,BELLEVUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,43200.0,NaN
1,180SEB,5.0,7497221.0,19483.0,0,43551.0,180.0,13451,SE 36TH ST,BELLEVUE,...,C,1.0,CLI1,NaN,102229.0,AM1,NaN,-1.0,44100.0,"V1,WK"
2,180SEB,5.0,7907553.0,23292.0,0,45017.0,180.0,3000,LANDERHOLM CIR SE,BELLEVUE,...,C,1.0,CLI1,NaN,103990.0,AM1,NaN,-1.0,45900.0,"CA,LI,V1"
3,180SEB,5.0,8215306.0,23394.0,1,45613.0,120.0,3843,166TH AVE SE,BELLEVUE,...,C,1.0,NaN,CLI1,103990.0,NaN,AM1,-1.0,-1.0,"CA,LI,V1"
4,180SEB,5.0,9138510.0,19589.0,1,46534.0,120.0,6082,167TH AVE SE,BELLEVUE,...,C,1.0,NaN,CLI1,102229.0,NaN,AM1,-1.0,-1.0,"V1,WK"


In [3]:
URID_location[["LON","LAT"]].head()

,LON,LAT
0,-122.182881,47.591642
1,-122.160469,47.579469
2,-122.147938,47.584411
3,-122.118146,47.575759
4,-122.116614,47.548175


In [4]:
urid_LAT = URID_location["LAT"][0]
urid_LON = URID_location["LON"][0]
print(urid_LAT)
print(urid_LON)

47.591642
-122.182881


In [5]:
total_times = []
out_start_points = []
out_end_points = []
in_total_time = []
in_start_points = []
in_end_points = []
osrm_url = "http://router.project-osrm.org/route/v1/driving/"
route_url = osrm_url + str(URID_location["LON"][1]) + "," + str(URID_location["LAT"][1]) + ";" + str(urid_LON) + "," + str(urid_LAT) + ";" + str(URID_location["LON"][2]) + "," + str(URID_location["LAT"][2]) + "?overview=false"
route_requests = requests.get(route_url)
route_results = route_requests.json()

In [6]:
route_results.keys()

[u'routes', u'code', u'waypoints']

In [7]:
route_url

'http://router.project-osrm.org/route/v1/driving/-122.160469,47.579469;-122.182881,47.591642;-122.147938,47.584411?overview=false'

In [8]:
route_results['routes']

[{u'distance': 12892.9,
  u'duration': 1014.7,
  u'legs': [{u'distance': 5854.8,
    u'duration': 484,
    u'steps': [],
    u'summary': u''},
   {u'distance': 7038.1, u'duration': 530.7, u'steps': [], u'summary': u''}]}]

In [9]:
route_results['routes'][0]['duration']

1014.7

In [10]:
lat1 = URID_location["LAT"][1]
lon1 = URID_location["LON"][1] 
lat2 = URID_location["LAT"][2] 
lon2 = URID_location["LON"][1]

In [11]:
def getOSRMOutput (lat1, lon1, lat2, lon2):

    """

    Args:
        lat1/lon1 (float): first lat/lon pair (start point)
        lat2/lon2 (float): second lat/lon pair (end point)

    Returns:
        json object containing data about traveling between two pairs
    """

    osrm_url = "http://router.project-osrm.org/route/v1/driving/"
    #route_url = osrm_url + str(lat1) + "," + str(lon1)
    #route_url = route_url + ";" + str(lat2) + "," + str(lon2) + "?overview=false"
    route_url = osrm_url + str(lon1) + "," + str(lat1)
    route_url = route_url + ";" + str(lon2) + "," + str(lat2) + "?overview=false"
    #print route_url
    route_requests = requests.get(route_url)
    route_results = route_requests.json()
    #print route_results
    return route_results



In [12]:
route_results_test = getOSRMOutput(lat1, lon1, lat2, lon2)

In [13]:
route_results_test

{u'code': u'Ok',
 u'routes': [{u'distance': 1590.2,
   u'duration': 149.5,
   u'legs': [{u'distance': 1590.2,
     u'duration': 149.5,
     u'steps': [],
     u'summary': u''}]}],
 u'waypoints': [{u'hint': u'Z898gGrPfIAHugsAXgAAACsBAAAxAAAAAAAAAKrVdQCr1XUA6uQAAKv6t_hJAdYCq_q3-E0B1gIBAAEBr60tjA==',
   u'location': [-122.160469, 47.579465],
   u'name': u'Southeast 36th Street'},
  {u'hint': u'Nh3JijodyYp6VQwADwAAADEAAAAAAAAAAAAAABQ3Ig4TNyIO6uQAAJ76t_gKEtYCq_q3-JsU1gIAAAEBr60tjA==',
   u'location': [-122.160482, 47.583754],
   u'name': u'Southeast 30th Street'}]}

In [14]:
total_dist_test = route_results_test[u'routes'][0][u'distance']
total_dist_test

1590.2

In [15]:
def mileage (lat1, lon1, lat2, lon2):

    """
    Used to find mileage for taxi function.

    Args:
        lat1/lon1 (float): first lat/lon pair (start point)
        lat2/lon2 (float): second lat/lon pair (end point)

    Returns:
        total street network distance between pairs
    """

    route_results = getOSRMOutput(lat1, lon1, lat2, lon2)
    total_dist = route_results[u'routes'][0][u'distance']
    #print total_dist
    return(total_dist)



In [16]:
milage_test = mileage(lat1, lon1, lat2, lon2)
milage_test

1590.2

In [17]:
def travel_time (lat1, lon1, lat2, lon2):

    """

    Args:
        lat1/lon1 (float): first lat/lon pair (start point)
        lat2/lon2 (float): second lat/lon pair (end point)

    Returns:
        total non traffic time it takes to get between two pairs of lat/lon
    """

    route_results = getOSRMOutput(lat1, lon1, lat2, lon2)
    total_time = route_results[u'routes'][0][u'duration']
    # print total_time
    return(total_time)


In [18]:
travel_test = travel_time(lat1, lon1, lat2, lon2)
travel_test

149.5